In [0]:
!pip install transformers torch accelerate

In [0]:
import mlflow
from transformers import pipeline

In [0]:
classifier = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english"
)

In [0]:
reviews = [
    "This product is amazing! I loved the quality.",
    "Terrible quality, waste of money.",
    "Decent product, but delivery was late.",
    "Excellent value for money!"
]

In [0]:
results = classifier(reviews)


In [0]:
for review, result in zip(reviews, results):
    print(f"Text: {review}")
    print(f"Prediction: {result['label']} | Score: {result['score']:.4f}")
    print("-" * 40)

In [0]:
with mlflow.start_run(run_name="sentiment_analysis_mosaic_ai"):

    # Log model info
    mlflow.log_param("task", "sentiment-analysis")
    mlflow.log_param("model_name", "distilbert-base-uncased-finetuned-sst-2")
    mlflow.log_param("num_reviews", len(reviews))

    # Example metric
    mlflow.log_metric(
        "sample_positive_ratio",
        sum(1 for r in results if r["label"] == "POSITIVE") / len(results)
    )
    # Log predictions as an artifact
    predictions = [
        {
            "text": review,
            "label": result["label"],
            "score": float(result["score"])
        }
        for review, result in zip(reviews, results)
    ]
    mlflow.log_dict(predictions, "sentiment_predictions.json")